# 加载库函数及导入文件

In [96]:
import pandas as pd
import pathlib
base_path = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
output_dir_path = pathlib.Path(r"E:\张楠\测试\output")
excel_file = pd.ExcelFile(base_path)
%run corelibs.py
excel_file.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

In [9]:
import yaml
with open('config.yaml', 'r', encoding='utf-8') as f:
    CONF_DATA = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
CONF_DATA

# 自定义函数及测试

In [98]:
df = parse_sheet_general(excel_file,0,CONF_DATA['天津银行']['流水'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13162 entries, 0 to 13161
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   日期          13162 non-null  object 
 1   时间          13162 non-null  object 
 2   姓名          13162 non-null  object 
 3   账号          13162 non-null  object 
 4   摘要          13161 non-null  object 
 5   币种          13162 non-null  object 
 6   出账金额        547 non-null    float64
 7   入账金额        12615 non-null  float64
 8   余额          13162 non-null  float64
 9   交易机构        9123 non-null   object 
 10  对方账号        12376 non-null  object 
 11  对方户名        12369 non-null  object 
 12  对方开户行       10565 non-null  object 
 13  IP地址        203 non-null    object 
 14  我方银行名称      13162 non-null  object 
 15  对方银行机构代码    10565 non-null  object 
 16  受理机构代码      9123 non-null   object 
 17  交易日期        13162 non-null  object 
 18  借贷标志        13162 non-null  object 
 19  交易金额        13162 non-nul

D:\soft\WPy64-3890\notebooks\pynbCaseData\corelibs.py:21: ParserWarning: Both a converter and dtype were specified for column 交易金额 - only the converter will be used
  df = excel_file.parse(sheet_name=sheet, header=0, skiprows=0, dtype=str, converters=converters)
D:\soft\WPy64-3890\notebooks\pynbCaseData\corelibs.py:21: ParserWarning: Both a converter and dtype were specified for column 余额 - only the converter will be used
  df = excel_file.parse(sheet_name=sheet, header=0, skiprows=0, dtype=str, converters=converters)


In [70]:
save_accounts(df, output_dir_path, '天津银行')

2

In [100]:
grouped = df.groupby('账号')
df_list = [x.reset_index(drop=True) for _ , x in grouped]
save_statements(df_list, output_dir_path, '天津银行',"hahahahahaaaa")

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:07<00:00,  4.36it/s]


13162

In [20]:
verify_data(df, CONF_DATA['天津银行']['账户']['verify_cols'])

False

In [ ]:
df_list

In [ ]:
open

# 执行

In [ ]:
cdid = CONF_DATA['天津银行']['流水']['CDid']
_C_crit = df[cdid['CD_col']] == cdid['C']
df[cdid['C_col']] = df[cdid['trans_col']][_C_crit]
df[cdid['D_col']] = df[cdid['trans_col']][~_C_crit]
df

In [ ]:
df = excel_file.parse(sheet_name=0, header=0, skiprows=0, nrows=5, dtype=str, converters={'交易金额':float})
df.info()

In [14]:
pd.to_datetime(df['交易日期']).dt.time

0        00:00:00
1        00:00:00
2        00:00:00
3        00:00:00
4        00:00:00
           ...   
13157    00:13:32
13158    00:15:25
13159    00:15:55
13160    00:26:51
13161    00:28:47
Name: 交易日期, Length: 13162, dtype: object

In [85]:
converters = {_col:float for _col in CONF_DATA['天津银行']['流水']['amount_cols']}
converters

{'交易金额': float, '余额': float}

# 杂项

In [45]:
base_path

WindowsPath('E:/张楠/2023赵益华927/银行/天津银行/滨监查询（2023）05217.xls')

In [93]:
with open(output_dir_path.joinpath('0查询文号.txt'),'a') as f:
    f.write('hahah')

In [54]:
'_'.join(['hahah',str (2)])

'hahah_2'